# ArcPy Cursors !

<ul>
    <li>Finding and extracting data</li>
    <li>Cleaning and Maintaining Table Data</li>
</ul>

<ul>
    <li>ArcPy Search Cursors</li>
    <li>ArcPy Update Cursors</li>
</ul>

In [ ]:
import os
import arcpy

Now that we have the data prepped how we want it, lets review the table.

<br>
<br>
We have done some geoprocessing, but how do we extract information and transform the table?
We use ArcPy Cursors!

- [arcpy.da.SearchCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/searchcursor-class.htm)
- [arcpy.da.InsertCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/insertcursor-class.htm)
- [arcpy.da.UpdateCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/updatecursor-class.htm)

<br>
ArcPy Cursors require two parameters:
<br> 1. Input Feature Class
<br> 2. A list of fields

In [ ]:
# lets first create a search cursor and just explore what the cursor looks like
# not the correct way of creating a cursor

fields = ["addr_city", "addr_state", "name", "rating", "nhood"]

cursor = arcpy.da.SearchCursor("sf_coffee_sp", fields)

In [ ]:
cursor

In [ ]:
for row in cursor:
    print(type(row), row)

########### "addr_city", "addr_state", "name", "rating", "nhood"

In [ ]:
type(cursor)

In [ ]:
row

In [ ]:
row[2]

In [ ]:
for row in cursor:
    if row[2] == "Peet's Coffee":
        print(row)

In [ ]:
# Once the iterator has been iterated through, 
# it needs to be reset before you can iterate again
# Using the reset method
cursor.reset()

In [ ]:
for row in cursor:
    if row[2] == "Peet's Coffee":
        print(row)

In [ ]:
row

In [ ]:
# The cursors are stored in the memory, 
# if you are working with large tables, you may want to delete them after you're done.
del row
del cursor

In [ ]:
cursor

In [ ]:
fields = ["addr_city", "addr_state", "name", "rating", "nhood"]
cursor = arcpy.da.SearchCursor("sf_coffee_sp", fields)

Wait, instead of having to delete the cursors and rows, we could instead just use
a "With" statement! <br>

This gives us the ability to temporarily save the cursor, no need to delete it after 
it's created.

<b> The below is the proper way to use a cursor: </b>

In [ ]:
with arcpy.da.SearchCursor("sf_coffee_sp", fields) as cursor:
    for row in cursor:
        print(row)

<b> Why are these useful? </b>

Imagine you had to get all of the values from the rating fields into a python list, <br>
you could do that by using a search cursor to extract that information.


In [ ]:
rating_values = []

with arcpy.da.SearchCursor("sf_coffee_sp", fields) as cursor:
    for row in cursor:
        rating_values.append(row[3])

In [ ]:
rating_values

<b> We have the values in a list now, but what if we just wanted to see the unique values? <b>
<br>
<b> What Python data type does not allow duplicates? </b>

In [ ]:
set(rating_values)

Is there a way we can create a repeatable process so that we could use this logic again with different data in the future?

<b> Notice we have "Good" and "Okay" within the column </b>
<br>
We want to clean these up so that the column is made up of all numbers.
<br>

Since the search cursors return tuples, we are not allowed to make any changes to them, since tuples in python are immutable.
<br>
However, ArcPy has another cursor method named UpdateCursor(), which will return lists.
<br>
This will allow us to update the table!

Before we make changes to the table, lets create a copy for backup:

In [ ]:
# Clear the selection, just in case any rows are selected!
arcpy.management.SelectLayerByAttribute("sf_coffee_sp", 
                                        "CLEAR_SELECTION")

arcpy.management.CopyFeatures("sf_coffee_sp", 
                              "sf_coffee_sp_copy")

We will make the changes to the copy version we just created

In [ ]:
with arcpy.da.UpdateCursor("sf_coffee_sp_copy", fields) as cursor:
    for row in cursor:
        print(row)

In [ ]:
fields = ["addr_city", "addr_state", "name", "rating", "nhood"]

with arcpy.da.UpdateCursor("sf_coffee_sp_copy", fields) as cursor:
    for row in cursor:
        if row[3] == 'Okay':
            row[3] = '3'
        elif row[3] == 'Good':
            row[3] = '4'

        cursor.updateRow(row)

Let's work together to fill in the logic within the below cursors!

In [ ]:
fields = ["addr_city", "addr_state", "name", "rating", "nhood"]

In [ ]:
# There is more cleaning we can do on this table!
# Lets update the addr_city field and the addr_state field

with arcpy.da.UpdateCursor("sf_coffee_sp_copy", fields) as cursor:
    for row in cursor:
        # update state column
        
        # update the city column

        cursor.updateRow(row)

In [ ]:
# lets create a new column using an update cursor that has the full address!
# lets also put the text in all caps

# first create a new field in the feature layer:
arcpy.management.AddField("sf_coffee_sp_copy", "Full_Address", "TEXT")

address_fields = ["addr_house", "addr_stree", "Full_Address"]

with arcpy.da.UpdateCursor("sf_coffee_sp_copy", address_fields) as cursor:
    for row in cursor:
        
        cursor.updateRow(row)

Why we should use cursors over selections and calculate fields!!

1. Much Faster
2. Code Clarity

In [ ]:
# how we would typically manually update the fields:

arcpy.management.SelectLayerByAttribute("sf_coffee_sp_copy", "NEW_SELECTION", "rating IN ('3', '4', '5') And full_address IS NULL", None)
arcpy.management.CalculateField("sf_coffee_sp_copy", "full_address", "'address doesnt exist'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

arcpy.management.SelectLayerByAttribute("sf_coffee_sp_copy", "NEW_SELECTION", "rating IN ('2', '1') And full_address IS NULL", None)
arcpy.management.CalculateField("sf_coffee_sp_copy", "new_field", "'test'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

arcpy.management.SelectLayerByAttribute("sf_coffee_sp_copy", "NEW_SELECTION", "rating IN ('2', '1', '3', '4') And full_address IS NOT NULL", None)
arcpy.management.CalculateField("sf_coffee_sp_copy", "new_field2", "'test'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")